<a href="https://colab.research.google.com/github/Niel-Reitmann/Disease-Neural-Network/blob/main/Disease_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
from csv import reader
import numpy as np
import random
from operator import itemgetter
from matplotlib import pyplot as plt

In [ ]:
data = open ("/gdrive/My Drive/COVID-19-geographic-disbtribution-worldwide-2020-04-14 (2).csv")
data = list(reader(data, delimiter=','))
print (data)

In [ ]:
#0, 3, 7, 8, 9
for row in data:
  del row[8]
  del row[7]
  del row[3]
  del row[0]


In [ ]:
data = data[1:]

In [ ]:
countries =[]
ndata= []
for row in data:
  a = row[:4]
  a.append(row[5])
  if row[4] in countries:
    ndata[countries.index(row[4])].append(a)
  else:
    countries.append(row[4])
    ndata.append([a])

print(ndata)



In [ ]:
how_much_data = []
for i in range(len(ndata)):
  how_much_data.append(len(ndata[i]))

plt.plot(sorted(how_much_data))
plt.show()

In [ ]:
if i in range (len(ndata)-1, -1, -1):
  if len(ndata[i]) < 80:
    del ndata[i]
    del countries [i]

print(len(ndata))

In [ ]:
def add(a):
  l = a.copy()
  for i in range(len(l)):
    if i != 0:
      l[i]=l[i]+l[i-1]
  return l


In [ ]:
def extract(country, column) :
  l =[]
  for i in country:
    l.append(int(i[column]))
  return l

In [ ]:
#Month between 1 and 12
#Day is between 1 and 31
#m*100+d
#Delete 1231 (dec 31st)

for country in ndata:
  for day in country:
    day.append(int(day[1])*100+int(day[0]))
    if day[-1] == 1231:
      del day





In [ ]:
for country in ndata:
  country.sort(key=itemgetter(5))


In [ ]:
mincase = 10000
sifted=[]
names=[]
for i in range(len(countries)):
  if max(add(extract(ndata[i], 2))) > mincase:
    names.append(countries[i])
    sifted.append(ndata[i])

print(names)
print(len(names))

In [ ]:
for i in range(len(names)):
  print(names[i])
  plt.clf()
  plt.plot(extract(sifted[i], 2))
  plt.plot(add(extract(sifted[i], 3)))
  plt.show()

In [ ]:
korea = sifted[names.index("South_Korea")]
china = sifted[names.index("China")]
Australia = ndata[countries.index("Australia")]

In [ ]:
ycc = []
ycd = []
ykc = []
ykd = []
yac = []
yad = []
for i in range(len(china)):
  ycc.append(int(china[i][2])/int(china[i][4])*30000)
  ycd.append(int(china[i][3])/int(china[i][4])*3000000)
  ykc.append(int(korea[i][2])/int(korea[i][4])*30000)
  ykd.append(int(korea[i][3])/int(korea[i][4])*3000000)
  yac.append(int(Australia[i][2])/int(Australia[i][4])*30000)
  yad.append(int(Australia[i][3])/int(Australia[i][4])*3000000)

In [ ]:
ycc = ycc[20:70]
ycd = ycd[20:70]
plt.plot(ycc)
plt.show()



In [ ]:
yac = yac[70:]
yad = yad[70:]
plt.plot(yac)
plt.plot(yad)
plt.show()


In [ ]:
ykc = ykc[45:]
ykd = ykd[45:]
plt.plot(ykc)
plt.show()

In [ ]:
def dataset(cases, deaths, population, num):
  xnew = []
  ynew = []
  for i in range(len(cases)-num):
    temp = []
    for a in range(num):
      temp.append(cases[i+a])
      temp.append(deaths[i+a])
    temp.append(int(population)/200000000)
    xnew.append(temp)
    ynew.append([cases[i+num], deaths[i+num]])

  return xnew, ynew

In [ ]:
dataset(yac, yad, int(Australia[0][4]), 13)[0]

In [ ]:
def data():
  xc, yc = dataset(ycc,ycd, china[0][4], 13)
  xk, yk = dataset(ykc,ykd, korea[0][4], 13)
  xa, ya = dataset(yac,yad, Australia[0][4], 13)
  x = xc+xk
  y = yc+yk
  temp = list(zip(x, y))
  random.shuffle(temp)
  x, y = zip(*temp)
  xtrain = x[:-18]
  ytrain = y[:-18]
  xtest = x[-18:]
  ytest = y[-18:]
  assert xtrain+xtest==x and ytrain+ytest==y
  return xtrain, xtest, ytrain, ytest

xtrain, xtest, ytrain, ytest = data()



In [ ]:
from tensorflow import keras

In [ ]:
model = keras.Sequential()
model.add(keras.layers.Dense(units=27, input_shape=(27,), activation="relu"))
model.add(keras.layers.Dense(units=64, activation="relu"))
model.add(keras.layers.Dropout(0.1))
model.add(keras.layers.Dense(units=128, activation="relu"))
model.add(keras.layers.Dropout(0.1))
model.add(keras.layers.Dense(units=128, activation="relu"))
model.add(keras.layers.Dropout(0.1))
model.add(keras.layers.Dense(units=64, activation="relu"))
model.add(keras.layers.Dropout(0.1))
model.add(keras.layers.Dense(units=2, activation="relu"))
model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.1), loss="mean_squared_error")


In [ ]:
model.fit(
    x = xtrain,
    y = ytrain,
    batch_size = 5,
    epochs=1000

)

In [ ]:
def visualize(model, infec, kill, name, reali, realkill, population, offset):
  for con in range(len(name)):
    infected = infec[con].copy()
    dead = kill[con].copy()
    temp = []
    for i in range(len(infected)):
      temp.append(infected[i])
      temp.append(dead[i])

    temp.append(population[con]/200000000)
    for _ in range(offset[con]):
      infected.insert(0,0)
      dead.insert(0,0)
    for i in range(100):
      p = model.predict(np.array([temp]))
      temp.pop(1)
      temp.pop(0)

      popu = temp.pop(-1)

      temp.append(p[0][0])
      temp.append(p[0][1])

      temp.append(popu)


      infected.append(p[0][0])
      dead.append(p[0][1])

    print(name[con])
    print("infected")
    plt.plot(infected, "r-")
    plt.plot(reali[con], "b--")
    plt.plot(infected[offset[con]:13+offset[con]], "k-")

    plt.show()


    print(name[con])
    print("dead")
    plt.plot(dead, "r-")
    plt.plot(realkill[con], "b--")
    plt.plot(dead[offset[con]:13+offset[con]], "k-")

    plt.show()

In [ ]:
visualize(model, [ycc[:13], ykc[:13], yac[:13]],
          [ycd[:13], ykd[:13], yad[:13]],
          ["China","South Korea","Australia"],
          [ycc, ykc, yac],
          [ycd,ykd, yad],
          [int(china[0][4]),int(korea[0][4]),int(Australia[0][4])],
          [0, 0, 0])